In [1]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyairports.airports import Airports
from datetime import datetime
import plotly.express as px
from datetime import datetime, timedelta

# Importing Dataset
data=pd.read_csv("Data/Flight_Data_Cleaned.csv")
data.head()

,searchDate,flightDate,Day_of_Flight,NumDays,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,Flight_time_in_minutes,isBasicEconomy,...,totalFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,Num_Segments,segmentsDepartureAirportCode,segmentsAirlineName,segmentsEquipmentDescription,segmentsCabinCode,cabin_code_weight
0,2022-05-29,2022-06-18,6,20,LAX,Los Angeles Intl,ATL,Hartsfield Jackson Atlanta Intl,259,0,...,1028.61,7,1943.0,ATL,1,LAX,Delta,Airbus A321,coach,1
1,2022-05-03,2022-06-24,5,52,CLT,Charlotte Douglas Intl,LAX,Los Angeles Intl,490,0,...,537.10,9,2171.0,ATL||LAX,3,CLT||ATL,Delta||Delta,Boeing 717||Airbus A321,coach||coach,2
2,2022-05-02,2022-06-30,4,59,LGA,La Guardia,ATL,Hartsfield Jackson Atlanta Intl,114,0,...,382.21,9,1700.0,MIA||ATL,3,LGA||MIA,Delta||Delta,Boeing 737-800||Boeing 737-900,coach||coach,2
3,2022-06-10,2022-08-04,4,55,LAX,Los Angeles Intl,SFO,San Francisco Intl,80,0,...,218.60,7,339.0,SFO,1,LAX,Alaska Airlines,Embraer 175,coach,1
4,2022-06-24,2022-07-20,3,26,LGA,La Guardia,DFW,Dallas Fort Worth Intl,401,1,...,207.60,9,1487.0,ATL||DFW,3,LGA||ATL,Delta||Delta,Airbus A320||Airbus A321,coach||coach,2


# EDA

##### Q1 - Date Range for Search Date and Flight Date

In [2]:
print(f"Min Search Date is: {data['searchDate'].min()} and Max Search Date is {data['searchDate'].max()}")

print(f"Min Flight Date is: {data['flightDate'].min()} and Max Flight Date is {data['flightDate'].max()}")


Min Search Date is: 2022-04-16 and Max Search Date is 2022-06-26
Min Flight Date is: 2022-04-17 and Max Flight Date is 2022-08-23


##### Q2 - Which is the busiest day???

In [3]:
days_counts = dict(data['Day_of_Flight'].value_counts())
days_counts = dict(sorted(days_counts.items()))

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

fig = px.bar(x=days, y=days_counts.values(), color=days_counts.values(),
             text=days_counts.values(), title="Day wise traffic",
             color_continuous_scale='aggrnyl',)

fig.update_layout(bargap=0.5)
fig.update_traces(textposition='outside')

# Center-align the title
fig.update_layout(title_x=0.5)

# Add x-axis and y-axis labels
fig.update_xaxes(title_text="Days")
fig.update_yaxes(title_text="Flight Count")
fig.update_layout(font=dict(color='black', size=9)) 
fig.show()


##### Q3 - Which is the busiest Airport???

In [14]:
#list of all airports:

starting_airport=list(data['starting_airport_name'].unique())
destination_airport=list(data['destination_airport_name'].unique())

all_airports=list(set(starting_airport+destination_airport))
all_airports

traffic_count={}
for airport in all_airports:
    airport_traffic_count=0
    for row in range(data.shape[0]):
        try:
            if data['starting_airport_name'][row]==airport:
                airport_traffic_count+=1
            elif data['destination_airport_name'][row]==airport:
                airport_traffic_count+=1
            else:
                continue
        except:
            print(ValueError)
            continue
    traffic_count[airport]=airport_traffic_count
traffic_count

{'Detroit Metro Wayne Co': 219911,
 'Philadelphia Intl': 227989,
 'Newark Liberty Intl': 206726,
 'La Guardia': 299884,
 'Charlotte Douglas Intl': 255145,
 'Miami Intl': 256210,
 'Metropolitan Oakland Intl': 155908,
 'Hartsfield Jackson Atlanta Intl': 257050,
 'Los Angeles Intl': 399256,
 'Chicago Ohare Intl': 266030,
 'San Francisco Intl': 281134,
 'Dallas Fort Worth Intl': 269559,
 'Denver Intl': 233059,
 'Washington Dulles Intl': 177799,
 'General Edward Lawrence Logan Intl': 287515,
 'John F Kennedy Intl': 206825}

In [19]:

traffic_count = dict(sorted(traffic_count.items(), key=lambda item: item[1]))



fig = px.bar(x=traffic_count.keys(), y=traffic_count.values(), color=traffic_count.values(),
             text=traffic_count.values(), title="Airport traffic traffic"
             ,template='plotly_dark')

fig.update_layout(bargap=0.5)
fig.update_traces(textposition='outside')

# Center-align the title
fig.update_layout(title_x=0.5)

# Add x-axis and y-axis labels
fig.update_xaxes(title_text="Airport Name")
fig.update_yaxes(title_text="Flight Count")
fig.show()

'''Los Angeles International airport is the busiest 
airport with 399256 flights landing or departing from this airport
in the period of 2022-04-16 and 2022-08-23
'''


##### Q4 - For top 5 busiest airports which was the busiest month??

In [38]:
top_5_busiest_airports=['Dallas Fort Worth Intl', 'San Francisco Intl', 
                        'General Edward Lawrence Logan Intl', 
                        'La Guardia', 'Los Angeles Intl']

start_date = datetime(2022, 4, 17)
end_date = datetime(2022, 8, 23)

month_year_list = []

current_date = start_date
while current_date <= end_date:
    month_year = current_date.strftime("%B %Y")
    month_year_list.append(month_year)
    current_date += timedelta(days=30)

for airport in top_5_busiest_airports:
    airport_month_year_traffic={}
    for month_year in month_year_list:
        count=0
        for row in range(data.shape[0]):
            
            if data['starting_airport_name'][row]==airport or data['destination_airport_name'][row]==airport:
                flight_month_year=datetime.strptime(data['flightDate'][row], '%Y-%m-%d').strftime("%B %Y")
                if flight_month_year==month_year:
                    count+=1
                else: continue
            else: continue
        airport_month_year_traffic[month_year]=count
    print(f"displaying results for {airport}")
    fig = px.bar(x=airport_month_year_traffic.keys(), y=airport_month_year_traffic.values(), color=airport_month_year_traffic.values(),
                text=airport_month_year_traffic.values(), title= f"{airport} Month Year traffic"
                ,template='plotly_dark')

    fig.update_layout(bargap=0.5)
    fig.update_traces(textposition='outside')

    # Center-align the title
    fig.update_layout(title_x=0.5)

    # Add x-axis and y-axis labels
    fig.update_xaxes(title_text= "Month Year")
    fig.update_yaxes(title_text="Flight Count")
    fig.show()

#June 2022 has been busiest for all airports

displaying results for Dallas Fort Worth Intl


displaying results for San Francisco Intl


displaying results for General Edward Lawrence Logan Intl


displaying results for La Guardia


displaying results for Los Angeles Intl


##### Q5 - How many airlines are there and Which airline has most flights?

In [55]:
distinct_airlines=[]

for flight in range(data.shape[0]):
    flight_segment=data['segmentsAirlineName'][flight].split('||')
    for segment in flight_segment:
        if segment not in distinct_airlines:
            distinct_airlines.append(segment)
        else:
            continue
print(f'Total Number of Airlines are : {len(distinct_airlines)}')
print(distinct_airlines)

'''
There are 14 distinct Airlines in the dataset
'''

Total Number of Airlines are : 14
['Delta', 'Alaska Airlines', 'United', 'American Airlines', 'Spirit Airlines', 'Southern Airways Express', 'JetBlue Airways', 'Cape Air', 'Frontier Airlines', 'Sun Country Airlines', 'Key Lime Air', 'Contour Airlines', 'Boutique Air', 'Hawaiian Airlines']


In [57]:
airline_flight_count={}
for airline in distinct_airlines:
    count=0
    for flight in range(data.shape[0]):
        flight_segment=data['segmentsAirlineName'][flight].split('||')
        for segment in flight_segment:
            if segment==airline:
                count+=1
            else:
                continue
    airline_flight_count[airline]=count

airline_flight_count = dict(sorted(airline_flight_count.items(), key=lambda item: item[1]))



fig = px.bar(x=airline_flight_count.keys(), y=airline_flight_count.values(), color=airline_flight_count.values(),
             text=airline_flight_count.values(), title="Airline Flight Count"
             ,template='plotly_dark')

fig.update_layout(bargap=0.5)
fig.update_traces(textposition='outside')

# Center-align the title
fig.update_layout(title_x=0.5)

# Add x-axis and y-axis labels
fig.update_xaxes(title_text="AirLine Name")
fig.update_yaxes(title_text="Flight Count")
fig.show()             

'''
United, Delta and American Airlines have the most flights
'''

##### Q6 - For the flight with max distance, does the number of days between search and flight date has any impact on the total fare??

In [104]:
data['totalTravelDistance'].max()
data[data['totalTravelDistance']==3958.0]['starting_airport_name'].unique()
data[data['totalTravelDistance']==3958.0]['destination_airport_name'].unique()

'''
'Max Distance for a flight in the data is between San Francisco Intl, Miami Intl 
and the distance is 3958 miles
'''
temp_data=data[data['totalTravelDistance']==3958.0]
temp_data.reset_index(inplace=True)

unique_date_range=sorted(list(temp_data['NumDays'].unique()))
unique_date_range_avg_fare={}


for date_range in unique_date_range:
    total_fare=0
    count=0
    for flight in range(temp_data.shape[0]):
        if temp_data['NumDays'][flight]==date_range:
            total_fare+=temp_data['totalFare'][flight]
            count+=1
    avg_fare=total_fare/count
    unique_date_range_avg_fare[date_range]=avg_fare


unique_date_range_avg_fare = dict(sorted(unique_date_range_avg_fare.items(), key=lambda item: item[1]))

fig = px.bar(x=list(unique_date_range_avg_fare.keys()), y=unique_date_range_avg_fare.values(), color=unique_date_range_avg_fare.values(),
             text=unique_date_range_avg_fare.values(), title="Difference between Search Date and Flight Date vs Average Fare"
             ,template='plotly_dark')

fig.update_layout(bargap=0.5)
fig.update_traces(textposition='outside')

# Center-align the title
fig.update_layout(title_x=0.5)

# Add x-axis and y-axis labels
fig.update_xaxes(title_text="Date Difference between search and flight date")
fig.update_yaxes(title_text="Average Fare")
fig.show()


'''
According to the graph, we can conclude that at number of days 
between search date and flight date = 27, the flight fare was minimum which is 367.21 USD
So, we should book flights a month in advance for the cheapest tickets.
'''

dict_keys([27, 46, 32, 16, 6, 14, 5, 11, 33, 2, 21, 9, 20, 15, 19, 4, 13, 10, 37, 40, 30, 31, 22, 41, 17, 52, 45, 25, 8, 12, 29, 60, 35, 3, 23, 56, 7, 28, 57, 1, 24, 53, 38, 55, 42, 18, 26, 47, 36, 48, 50, 43, 39, 49, 44])


##### ADD distances which have null

In [45]:
data[data['totalTravelDistance'].isna()]

,searchDate,flightDate,Day_of_Flight,NumDays,startingAirport,starting_airport_name,destinationAirport,destination_airport_name,Flight_time_in_minutes,isBasicEconomy,...,totalFare,seatsRemaining,totalTravelDistance,segmentsArrivalAirportCode,Num_Segments,segmentsDepartureAirportCode,segmentsAirlineName,segmentsEquipmentDescription,segmentsCabinCode,cabin_code_weight
18,2022-05-13,2022-07-08,5,56,DFW,Dallas Fort Worth Intl,EWR,Newark Liberty Intl,496,0,...,275.58,0,NaN,MCO||EWR,3,DFW||MCO,Spirit Airlines||Spirit Airlines,AIRBUS INDUSTRIE A320 SHARKLETS||Airbus A319,coach||coach,2
27,2022-05-13,2022-06-11,6,29,LGA,La Guardia,PHL,Philadelphia Intl,414,0,...,366.60,7,NaN,ORD||PHL,3,LGA||ORD,American Airlines||American Airlines,Boeing 737-800||Airbus A321,coach||coach,2
29,2022-04-27,2022-05-10,2,13,CLT,Charlotte Douglas Intl,DTW,Detroit Metro Wayne Co,579,0,...,244.08,0,NaN,MCO||DTW,3,CLT||MCO,Spirit Airlines||Spirit Airlines,AIRBUS INDUSTRIE A321 SHARKLETS||AIRBUS INDUST...,coach||coach,2
31,2022-04-16,2022-04-21,4,5,LAX,Los Angeles Intl,DTW,Detroit Metro Wayne Co,260,0,...,270.59,0,NaN,DTW,1,LAX,Spirit Airlines,AIRBUS INDUSTRIE A320 SHARKLETS,coach,1
68,2022-05-27,2022-07-15,5,49,DTW,Detroit Metro Wayne Co,LAX,Los Angeles Intl,410,0,...,239.58,0,NaN,LAS||LAX,3,DTW||LAS,Spirit Airlines||Spirit Airlines,AIRBUS INDUSTRIE A321 SHARKLETS||,coach||coach,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999939,2022-06-18,2022-07-10,7,22,LGA,La Guardia,DTW,Detroit Metro Wayne Co,120,0,...,130.59,0,NaN,DTW,1,LGA,Spirit Airlines,Airbus A319,coach,1
1999945,2022-05-05,2022-06-01,3,27,EWR,Newark Liberty Intl,ATL,Hartsfield Jackson Atlanta Intl,139,0,...,42.79,0,NaN,ATL,1,EWR,Spirit Airlines,AIRBUS INDUSTRIE A321 SHARKLETS,coach,1
1999955,2022-05-25,2022-07-08,5,44,BOS,General Edward Lawrence Logan Intl,DTW,Detroit Metro Wayne Co,247,1,...,187.20,9,NaN,IAD||DTW,3,BOS||IAD,United||United,Boeing 737 MAX 9||Boeing 737-700,coach||coach,2
1999964,2022-05-18,2022-05-24,2,6,PHL,Philadelphia Intl,DTW,Detroit Metro Wayne Co,97,0,...,181.68,0,NaN,MIA||DTW,3,PHL||MIA,Spirit Airlines||Spirit Airlines,Airbus A319||Airbus A319,coach||coach,2
